In [37]:
import datetime
import sys
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# 读取数据
data  = pd.read_csv('/Users/zengyan/Excelsior/ai-trader/order_book_ai/data/okxorderbook20240421_1_cleaned_featured_targeted.csv')
# 去除空值
data = data.dropna()
data

,datetime,server_time,ask_1_price,ask_2_price,ask_3_price,bid_3_price,bid_2_price,bid_1_price,ask_1_amount,ask_2_amount,...,w70_20_10_ask_over_bid,w70_20_10_ask_bid_gap,w80_0_20_ask_over_bid,w80_0_20_ask_bid_gap,w80_10_10_ask_over_bid,w80_10_10_ask_bid_gap,w90_0_10_ask_over_bid,w90_0_10_ask_bid_gap,is_up,is_down
55,2024-04-20 16:00:55,1713628855004,64895.5,64895.6,64895.7,64894.4,64894.7,64895.4,0.360907,0.237559,...,0.426108,-0.402418,0.376789,-0.452655,0.387891,-0.441035,0.358109,-0.472636,0.0,0.0
56,2024-04-20 16:00:56,1713628856004,64895.5,64895.6,64895.7,64894.4,64894.7,64895.4,0.360907,0.237559,...,0.809774,-0.105110,0.716756,-0.164988,0.737876,-0.150830,0.681747,-0.189240,0.0,0.0
57,2024-04-20 16:00:57,1713628857004,64895.5,64895.6,64895.7,64894.7,64894.9,64895.4,0.774021,0.237559,...,0.929451,-0.036564,0.874022,-0.067223,0.886641,-0.060085,0.853270,-0.079174,0.0,0.0
58,2024-04-20 16:00:58,1713628858004,64895.5,64895.6,64895.7,64894.7,64894.9,64895.4,0.391151,0.237559,...,1.011638,0.005785,0.903216,-0.050853,0.927945,-0.037374,0.862566,-0.073787,0.0,0.0
59,2024-04-20 16:00:59,1713628859004,64895.5,64895.6,64895.7,64894.7,64894.9,64895.4,0.671261,0.237559,...,1.852156,0.298776,1.729170,0.267177,1.757763,0.274774,1.683976,0.254837,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2024-04-21 04:00:38,1713672038006,65159.3,65159.4,65159.9,65156.0,65157.5,65158.4,1.459707,0.228647,...,18.710398,0.898531,19.812003,0.903902,20.847938,0.908458,22.983631,0.916610,0.0,0.0
138,2024-04-21 04:00:39,1713672039006,65159.3,65159.4,65159.9,65158.0,65158.4,65159.2,0.953097,0.203400,...,1.437446,0.179469,1.378652,0.159188,1.410336,0.170240,1.388695,0.162723,0.0,0.0
139,2024-04-21 04:00:40,1713672040006,65166.5,65166.6,65166.8,65164.1,65165.9,65166.4,0.368585,0.290000,...,0.471583,-0.359080,0.479413,-0.351888,0.426366,-0.402165,0.390376,-0.438460,0.0,0.0
140,2024-04-21 04:00:41,1713672041006,65166.5,65166.6,65166.8,65164.1,65165.9,65166.4,0.053493,0.290000,...,0.149080,-0.740523,0.150581,-0.738252,0.113577,-0.796014,0.085502,-0.842466,0.0,0.0


In [38]:

target = data['is_up']
only_features = data.drop(['is_up','datetime','server_time','is_down'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(only_features,target, test_size=0.4, random_state=42)
print("训练集特征：")
print(x_train.shape)
print("\n训练集目标：")
print(y_train.shape)
print("\n测试集特征：")
print(x_test.shape)
print("\n测试集目标：")
print(y_test.shape)


训练集特征：
(52, 134)

训练集目标：
(52,)

测试集特征：
(35, 134)

测试集目标：
(35,)


In [39]:
# 为了自动处理超参数
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_features='sqrt', random_state=1, n_jobs=-1)

# 这里有更合适的参数么？
#  oob_score 用于
param_grid = {
                "criterion": ['gini','entropy'],
                "min_samples_leaf": [1, 5, 10, 25],
                'min_samples_split': [2, 4, 10, 12, 16, 18, 25, 35],
                "n_estimators": [25, 50,100, 150], 
                'max_depth': [3, 5, 15, 25, 30],
                'max_features': ['sqrt'],
                'oob_score': [True]
            }


gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
gs = gs.fit(x_train, y_train)
print(gs.best_params_)
print(gs.best_score_)

{'criterion': 'gini', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 25, 'oob_score': True}
1.0


In [40]:
# 创建一个新的随机森林对象，放入最佳参数, 重新fit
best_params = gs.best_params_
rf2 = RandomForestClassifier(criterion=best_params['criterion'], 
                             n_estimators=best_params['n_estimators'],
                             min_samples_split=best_params['min_samples_split'],
                             min_samples_leaf=best_params['min_samples_leaf'],
                             max_features=best_params['max_features'],
                             max_depth=best_params['max_depth'],
                             oob_score=best_params['oob_score'],
                             random_state=1,
                             n_jobs=-1)
rf2.fit(x_train, y_train)
df1 = pd.DataFrame(x_train.columns,columns=['Variable'])
df2 = pd.DataFrame(rf2.feature_importances_,columns=['Importance'])
pd.concat([df1,df2],axis=1).sort_values(by='Importance',ascending=False)

,Variable,Importance
0,ask_1_price,0.0
84,w30_30_40_ask_over_bid,0.0
98,w40_30_30_ask_over_bid,0.0
97,w40_20_40_ask_bid_gap,0.0
96,w40_20_40_ask_over_bid,0.0
...,...,...
39,w0_70_30_ask_bid_gap,0.0
38,w0_70_30_ask_over_bid,0.0
37,w0_60_40_ask_bid_gap,0.0
36,w0_60_40_ask_over_bid,0.0


In [41]:
from sklearn import metrics
# 正式预测
pred = rf2.predict(x_test)
# 与真实值比较
print(metrics.accuracy_score(y_test,pred))


1.0
